In [23]:
# import modules

# etc
from typing import Tuple, List  # function annotation
from IPython.display import display

# basic data engineering
import numpy as np
import pandas as pd

# word embedding : dtm, tfidf
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# calculate distance of matrix
from scipy.spatial.distance import pdist
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances

In [24]:
# load data
df = pd.read_csv('../documents/data/movies_metadata.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [25]:
# check data
display(df.head(3))
display(df.columns)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

줄거리의 유사성을 찾고자 하므로 overview만 추출한다.

In [26]:
df_overview = pd.DataFrame(df[['title', 'overview']])[:2000]
df_overview

,title,overview
0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,Jumanji,When siblings Judy and Peter discover an encha...
2,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...
1995,Tex,Coming-of-age adventure about two teenage brot...
1996,Tron,As Kevin Flynn searches for proof that he inve...
1997,Swing Kids,The story of a close-knit group of young kids ...
1998,Halloween: H20,Two decades after surviving a massacre on Octo...


데이터 전처리

In [27]:
# NaN값 확인
display(df_overview.isnull().sum())

# 제목은 id가 있으니 무시해도 overview는 954개나 되므로 NaN값은 우선 0처리
df_overview['overview'].fillna("", inplace=True)
display(df_overview.isnull().sum())

title        0
overview    16
dtype: int64

title       0
overview    0
dtype: int64

In [28]:
# 일괄 소문자 변경 & 개수 조정
df_overview['overview'] = df_overview['overview'].str.lower()

빈도 확인

In [29]:
# dtm
dtm: np.ndarray = CountVectorizer().fit_transform(df_overview['overview']).toarray()
dtm.shape

# 데이터 개수가 2000개일 때 사용된 단어의 개수는 14195개다.

(2000, 14195)

In [30]:
# tfidf
tfidf: np.ndarray = TfidfVectorizer().fit(df_overview['overview']).transform(df_overview['overview']).toarray()
tfidf.shape

(2000, 14195)

In [31]:
sims_cosine = cosine_similarity(dtm)
sims_cosine_tfidf = cosine_similarity(tfidf)  

In [37]:
len(sims_cosine[0])
len(sims_cosine_tfidf[0])

2000

In [33]:
sims_cosine_tfidf[0]

array([1.        , 0.03454759, 0.01481731, ..., 0.01949542, 0.0097847 ,
       0.03464374])

In [34]:
dists_manhattan_tfidf = manhattan_distances(tfidf)
print(dists_manhattan_tfidf.shape)

(2000, 2000)


In [35]:
dists_euclidean_tfidf = euclidean_distances(tfidf)  # this may take a while... not efficient at all? cosine - it didn't take that much of a time. but manhattan... well that took some time.
print(dists_euclidean_tfidf.shape)  # (num_doc, num_doc)

(2000, 2000)


In [36]:
def get_recommendations(title: str,
                        # dists X
                        sims: np.ndarray,
                        indices: np.ndarray,
                        data: pd.DataFrame) -> List[Tuple[str, float]]:
    global TOP_N
    ### TODO 2 ###
    # 루프는 사용 X
    # use: np.argsort()[::-1] to recommend titles with scores
    titles_sorted: List[str] = np.argsort()
    sims_to_title_sorted: List[float] = ...
    results: List[Tuple[str, float]] = [
        (title, score)
        for title, score in zip(titles_sorted, sims_to_title_sorted)
    ]
    ##############
    return results[:TOP_N]